**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [21]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json
import random as rd
from collections import deque

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten, Dropout

# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [2]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e
    
    def get_epsilon(self):
        return self.epsilon

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

__Answer 1__:


The function act allows the agent to take an action given an observation of the current state of the environment. In the case where there are several actions and we can randomly choose one of them. If we ever happen upon a decent option, we’ll always choose that one in the future, even if there is a way better option available. To overcome this, we use the notion of __exploration__. The problem is that we even after we’ve explored all the options and we know for sure the best option, we still  choose a random action; the exploration doesn’t turn off. 

The standard way to get exploration is to introduce an additional term, __epsilon__. We then randomly generate a value, and if that value is less than epsilon, a random action is chosen, otherwise we follow the normal tactic of choosing the max Q value, in other words we use the notion of __exploitation__.

$$ a^{*} = argmax_{a} Q^{\pi} (s , a) $$

So, epsilon manage the tradeoff between exploration and exploitation thus : 

- During the exploration phase, the agent uses an epsilon greedy policy. Given a value for epsilon, the agent has a probability of epsilon to choose a random action. Otherwise, the agent will choose the action that has the best reward. This policy allows the agent to explore new possibilities than the one he already consider as the best, wich allows to discover new strategies.

- During the exploitation phase, we consider that the agent has explored enough stategies, we stop the epsilon greedy policy and then the agent will always select the action with the best reward.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [3]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [4]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=20 # set small when debugging
epochs_test=20 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.


__Answer 2__

```position``` is the array that contains the position of the rat on the board at a given time. The array is updated for each call of the function __act__  by setting to zeros all the elements except the position of the rat that is set to __1__ and the boarders to __-1__. The goal of the array is to keep track of the position of the mouse and give information about its playground. 

```board``` is the array that consists of the board on which the rat moves around, it's a view of the rewards.
Each of it cells is either neutral or represents the presence of the cheese or the poisonous cells. There informations represents the reward of each action and are represented by respectively 0, 0.5 and -1. 
The distribution of the reward (0.5 and -1) follows a binomial law that is defined is the __reset__ function. 

The environment send to the rat a partial view of ```position``` and  ```board``` (visibility of 2 cells from its current position : self.x and self.y)

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return rd.randint(0,3)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [5]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.learned_act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 9.5/10.0. Average score (-0.5)
Win/lose count 9.5/12.0. Average score (-1.5)
Win/lose count 9.5/23.0. Average score (-5.5)
Win/lose count 8.0/8.0. Average score (-4.125)
Win/lose count 11.5/20.0. Average score (-5.0)
Win/lose count 10.5/11.0. Average score (-4.25)
Win/lose count 11.0/12.0. Average score (-3.78571428571)
Win/lose count 5.0/14.0. Average score (-4.4375)
Win/lose count 10.0/15.0. Average score (-4.5)
Win/lose count 14.5/25.0. Average score (-5.1)
Win/lose count 11.0/7.0. Average score (-4.27272727273)
Win/lose count 8.0/12.0. Average score (-4.25)
Win/lose count 10.0/10.0. Average score (-3.92307692308)
Win/lose count 7.0/17.0. Average score (-4.35714285714)
Win/lose count 11.0/16.0. Average score (-4.4)
Win/lose count 4.5/7.0. Average score (-4.28125)
Win/lose count 9.0/12.0. Average score (-4.20588235294)
Win/lose count 10.5/14.0. Average score (-4.16666666667)
Win/lose count 15.5/16.0. Average score (-3.97368421053)
Win/lose count 9.5/13.0. Average score

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




__Answer 5 __
The Q-function is defined such that:



The action-value function can be written as
<img src="proof1.png" style="width:550px">

Thus
\begin{equation*}
Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

By using the Markov property and defining the function as a Bellman equation we have that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}


One assumes that the optimal policy $\pi^*$ exists, the optimal Q function is defined as:

\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> 
\end{equation*}

\begin{equation*}
Q^*(s,a)= \max_{\pi} E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] \> 
\end{equation*}

\begin{equation*}
Q^*(s,a)= E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma Q^{\pi^*}(s',a')] \> 
\end{equation*}

\begin{equation*}
Q^*(s,a)= E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}

Finally, a plausible objective is to minimize the loss function defined as:

\begin{equation*}
\mathcal{L}(\theta) = E_{s' \sim \pi^*(.|s,a)}( ( Q^* (s,a,\theta ) - Q(s,a,\theta)) ^2) 
\end{equation*}

\begin{equation*}
\mathcal{L}(\theta) =  E_{s' \sim \pi^*(.|s,a)} ( ( r(s,a)+\gamma\max_{a'}Q^{*}(s',a', \theta) - Q(s,a,\theta)) ^2) 
\end{equation*}

\begin{equation*}
\mathcal{L}(\theta) = E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [6]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = deque(maxlen=self.max_memory)        

    def remember(self, m):
        # move m = list(s_t, s_(t+1), a_t, r_t, game_over)
        self.memory.append(m)

    def random_access(self):
        idx = rd.randint(0,len(self.memory)-1)
        return self.memory[idx]

***
The pipeline we will use for training is given below:

In [7]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
    print('Final score: '+str(score/epoch))

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [8]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size
    

    def learned_act(self, s):
        
        s = np.reshape(s,[1,5,5,self.n_state])
        # Predict the reward value based on the given state
        act_values = self.model.predict(s)
        return np.argmax(act_values[0])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        
        s_ = np.reshape(s_,[1,5,5,self.n_state])
        n_s_ = np.reshape(n_s_,[1,5,5,self.n_state])
        
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))

        for i in range(self.batch_size):
            target = r_
            minibatch = self.memory.random_access()
            
            s_= minibatch[0]
            n_s_= minibatch[1]
            a_ = minibatch[2]
            r_= minibatch[3]
            game_over_= minibatch[4]
            
            input_states[i]= s_
            
            if game_over_:
                target= r_

            else:               
                target = (r_ + self.discount * np.amax(self.model.predict(n_s_)[0]))
                
            target_f = self.model.predict(s_)
            target_f[0][a_] = target
            self.model.fit(s_, target_f, epochs=1, verbose=0)
            
            target_q[i]=target_f
            
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            

class DQN_FC(DQN):
    def __init__(self, lr=0.1,  *args, **kwargs):
        super(DQN_FC, self).__init__(*args,**kwargs)
        
        # NN Model
        state_size = 50
        
        model = Sequential()
        model.add(Reshape((state_size,), input_shape=(5,5,self.n_state)))
        model.add(Dense(512, input_dim=state_size, activation='relu'))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(4, activation='linear'))
        #print(model.summary())
        #model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        model.compile(loss='mse', optimizer=Adam(lr=lr))
        self.model = model
        
        

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(lr=0.1, grid_size=size, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train0.mp4'))

Epoch 000/020 | Loss 29164.7383 | Win/lose count 2.0/4.0 (-2.0)
Epoch 001/020 | Loss 115.4741 | Win/lose count 3.5/5.0 (-1.5)
Epoch 002/020 | Loss 0.2150 | Win/lose count 5.5/9.0 (-3.5)
Epoch 003/020 | Loss 0.0858 | Win/lose count 4.5/7.0 (-2.5)
Epoch 004/020 | Loss 0.0694 | Win/lose count 12.0/25.0 (-13.0)
Epoch 005/020 | Loss 0.3616 | Win/lose count 7.0/7.0 (0.0)
Epoch 006/020 | Loss 0.1292 | Win/lose count 3.0/8.0 (-5.0)
Epoch 007/020 | Loss 0.1166 | Win/lose count 5.0/9.0 (-4.0)
Epoch 008/020 | Loss 0.0283 | Win/lose count 5.5/11.0 (-5.5)
Epoch 009/020 | Loss 0.4636 | Win/lose count 7.5/8.0 (-0.5)
Epoch 010/020 | Loss 0.1753 | Win/lose count 7.5/9.0 (-1.5)
Epoch 011/020 | Loss 0.3340 | Win/lose count 10.0/17.0 (-7.0)
Epoch 012/020 | Loss 0.4331 | Win/lose count 6.0/11.0 (-5.0)
Epoch 013/020 | Loss 0.2290 | Win/lose count 7.0/8.0 (-1.0)
Epoch 014/020 | Loss 0.3135 | Win/lose count 7.0/12.0 (-5.0)
Epoch 015/020 | Loss 0.6157 | Win/lose count 10.0/16.0 (-6.0)
Epoch 016/020 | Loss 0.39

In [19]:
# We test different learning rates
rates= [0.01, 0.05, 0.1, 0.2]
for rate in rates:
    print("\n rate : {0}".format(rate)  )
    env = Environment(grid_size=size, max_time=T, temperature=0.3)
    agent = DQN_FC(lr=rate, grid_size= size,  epsilon = 0.1, memory_size=2000, batch_size = 32)
    train(agent, env, 5, prefix='fc_train')


 rate : 0.01
Epoch 000/005 | Loss 14.5782 | Win/lose count 6.0/7.0 (-1.0)
Epoch 001/005 | Loss 4.5255 | Win/lose count 3.5/2.0 (1.5)
Epoch 002/005 | Loss 0.1347 | Win/lose count 7.5/9.0 (-1.5)
Epoch 003/005 | Loss 0.0149 | Win/lose count 4.5/6.0 (-1.5)
Epoch 004/005 | Loss 0.0016 | Win/lose count 7.5/12.0 (-4.5)
Final score: -1.4

 rate : 0.05
Epoch 000/005 | Loss 108.3145 | Win/lose count 2.5/7.0 (-4.5)
Epoch 001/005 | Loss 0.0084 | Win/lose count 7.5/10.0 (-2.5)
Epoch 002/005 | Loss 0.0483 | Win/lose count 5.5/8.0 (-2.5)
Epoch 003/005 | Loss 0.0395 | Win/lose count 11.5/23.0 (-11.5)
Epoch 004/005 | Loss 0.0462 | Win/lose count 6.5/7.0 (-0.5)
Final score: -4.3

 rate : 0.1
Epoch 000/005 | Loss 3710.3174 | Win/lose count 5.0/7.0 (-2.0)
Epoch 001/005 | Loss 94.5975 | Win/lose count 2.0/3.0 (-1.0)
Epoch 002/005 | Loss 115.3315 | Win/lose count 4.5/6.0 (-1.5)
Epoch 003/005 | Loss 273.5267 | Win/lose count 5.0/9.0 (-4.0)
Epoch 004/005 | Loss 151.0632 | Win/lose count 2.0/5.0 (-3.0)
Final 

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [9]:
class DQN_CNN(DQN):
    def __init__(self, lr=0.1, *args,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        filters = 32
        kernel_size = (2,2)
        
        model = Sequential()
        model.add(Conv2D(filters, kernel_size, input_shape=(5,5,self.n_state), activation='relu'))
        model.add(Conv2D(filters, kernel_size, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Flatten())
        model.add(Dense(4, activation='linear'))
        #print(model.summary())
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
       

In [17]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(lr=.1, grid_size = size, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train0.mp4'))

Epoch 000/020 | Loss 0.0172 | Win/lose count 2.5/2.0 (0.5)
Epoch 001/020 | Loss 0.0421 | Win/lose count 3.5/4.0 (-0.5)
Epoch 002/020 | Loss 0.0182 | Win/lose count 8.0/2.0 (6.0)
Epoch 003/020 | Loss 0.0167 | Win/lose count 4.5/4.0 (0.5)
Epoch 004/020 | Loss 0.0202 | Win/lose count 7.0/6.0 (1.0)
Epoch 005/020 | Loss 0.1698 | Win/lose count 14.0/3.0 (11.0)
Epoch 006/020 | Loss 0.1548 | Win/lose count 13.5/6.0 (7.5)
Epoch 007/020 | Loss 0.2449 | Win/lose count 23.0/2.0 (21.0)
Epoch 008/020 | Loss 0.4458 | Win/lose count 24.0/3.0 (21.0)
Epoch 009/020 | Loss 0.1848 | Win/lose count 23.0/6.0 (17.0)
Epoch 010/020 | Loss 0.8769 | Win/lose count 20.5/5.0 (15.5)
Epoch 011/020 | Loss 0.7117 | Win/lose count 20.5/3.0 (17.5)
Epoch 012/020 | Loss 0.5058 | Win/lose count 16.5/4.0 (12.5)
Epoch 013/020 | Loss 0.4779 | Win/lose count 10.5/4.0 (6.5)
Epoch 014/020 | Loss 0.8960 | Win/lose count 18.5/10.0 (8.5)
Epoch 015/020 | Loss 0.2182 | Win/lose count 15.5/3.0 (12.5)
Epoch 016/020 | Loss 0.4255 | Win/l

In [20]:
# we test different learning rates
rates= [0.01, 0.05, 0.1, 0.2]
for rate in rates:
    print("\n rate : {0}".format(rate)  )   
    env = Environment(grid_size=size, max_time=T, temperature=0.3)
    agent = DQN_CNN(lr=rate, grid_size= size, epsilon = 0.1, memory_size=2000, batch_size = 32)
    train(agent,env,5,prefix='cnn_train')


 rate : 0.01
Epoch 000/005 | Loss 0.0017 | Win/lose count 1.5/3.0 (-1.5)
Epoch 001/005 | Loss 0.0053 | Win/lose count 2.5/3.0 (-0.5)
Epoch 002/005 | Loss 0.0033 | Win/lose count 5.0/3.0 (2.0)
Epoch 003/005 | Loss 0.0010 | Win/lose count 4.0/3.0 (1.0)
Epoch 004/005 | Loss 0.0067 | Win/lose count 5.5/6.0 (-0.5)
Final score: 0.1

 rate : 0.05
Epoch 000/005 | Loss 0.0016 | Win/lose count 1.5/4.0 (-2.5)
Epoch 001/005 | Loss 0.0025 | Win/lose count 5.0/5.0 (0.0)
Epoch 002/005 | Loss 0.0453 | Win/lose count 1.0/5.0 (-4.0)
Epoch 003/005 | Loss 0.0611 | Win/lose count 3.5/0 (3.5)
Epoch 004/005 | Loss 0.1203 | Win/lose count 7.0/1.0 (6.0)
Final score: 0.6

 rate : 0.1
Epoch 000/005 | Loss 0.0017 | Win/lose count 2.5/5.0 (-2.5)
Epoch 001/005 | Loss 0.0049 | Win/lose count 4.0/1.0 (3.0)
Epoch 002/005 | Loss 0.0141 | Win/lose count 5.0/2.0 (3.0)
Epoch 003/005 | Loss 0.0714 | Win/lose count 5.0/1.0 (4.0)
Epoch 004/005 | Loss 0.1123 | Win/lose count 8.0/7.0 (1.0)
Final score: 1.7

 rate : 0.2
Epoch 

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [19]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(lr=.1, grid_size=size, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(lr=.1, grid_size=size, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('\nTest of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('\nTest of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')


Test of the CNN
Win/lose count 25.0/2.0. Average score (23.0)
Win/lose count 14.0/1.0. Average score (18.0)
Win/lose count 6.5/0. Average score (14.1666666667)
Win/lose count 13.5/0. Average score (14.0)
Win/lose count 16.0/0. Average score (14.4)
Win/lose count 13.5/1.0. Average score (14.0833333333)
Win/lose count 15.0/3.0. Average score (13.7857142857)
Win/lose count 17.5/0. Average score (14.25)
Win/lose count 7.0/0. Average score (13.4444444444)
Win/lose count 3.5/1.0. Average score (12.35)
Win/lose count 15.5/2.0. Average score (12.4545454545)
Win/lose count 5.5/0. Average score (11.875)
Win/lose count 12.0/0. Average score (11.8846153846)
Win/lose count 11.0/1.0. Average score (11.75)
Win/lose count 9.5/0. Average score (11.6)
Win/lose count 10.0/0. Average score (11.5)
Win/lose count 6.5/0. Average score (11.2058823529)
Win/lose count 21.0/0. Average score (11.75)
Win/lose count 16.5/1.0. Average score (11.9473684211)
Win/lose count 6.5/1.0. Average score (11.625)
Final score:

In [20]:
HTML(display_videos('cnn_test10.mp4'))

In [21]:
HTML(display_videos('fc_test10.mp4'))

- __Observations__:  the agent doesn't explore other areas and most of the time it sticks to the current board, so we have a low exploration rate. However, we can notice that the results given by the CNN are better than the ones given by the FC model


- __By changing the temperature__ : we are actually changing the probability of the binomial low. We notice that we improve the final score, but it is biased because by doing so we increase the probability of having cheese and we lower the probility of poisonious cells


***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [10]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            
            # Epsilon-greedy 
            agent.set_epsilon(np.float(agent.get_epsilon())*0.995)
            
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
            
        # Save as a mp4       
        env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
    print('Final score: '+str(score/epoch))
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size + 4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature
        
        self.board = np.zeros([grid_size,grid_size])
        self.position = np.zeros([grid_size,grid_size])
        self.malus_position = np.zeros([grid_size,grid_size])
        
        # coordinate of the rat
        self.x = 0
        self.y = 1
        
        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))
        
    def get_t(self):
        return self.t
    
    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b
             
        
    def act(self, action, train=True):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
        
        self.t = self.t + 1
        
        ## In Environment exploring:
        # You will have to change n_state to 3 because you will use one more layer!
        reward = 0
        if train:
            reward -= self.malus_position[self.x, self.y]

        self.malus_position[self.x, self.y] = 0.01
        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time

        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]
        
        return state, reward, game_over
    
    
    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
       
        self.malus_position[self.x, self.y] = 0

        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        
        return state

In [11]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(lr=.1, grid_size=size,  epsilon = 1, memory_size=16000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')

Epoch 000/020 | Loss 0.0721 | Win/lose count 7.5/14.44 (-6.94)
Epoch 001/020 | Loss 0.1250 | Win/lose count 5.96/7.66 (-1.7)
Epoch 002/020 | Loss 0.1701 | Win/lose count 11.85/3.71 (8.14)
Epoch 003/020 | Loss 0.3154 | Win/lose count 21.2/2.53 (18.67)
Epoch 004/020 | Loss 0.3159 | Win/lose count 15.73/2.63 (13.1)
Epoch 005/020 | Loss 0.7005 | Win/lose count 17.25/2.64 (14.61)
Epoch 006/020 | Loss 0.3867 | Win/lose count 18.16/2.61 (15.55)
Epoch 007/020 | Loss 0.6885 | Win/lose count 20.14/3.56 (16.58)
Epoch 008/020 | Loss 0.5714 | Win/lose count 16.69/2.66 (14.03)
Epoch 009/020 | Loss 0.6887 | Win/lose count 15.21/2.67 (12.54)
Epoch 010/020 | Loss 0.7056 | Win/lose count 18.15/1.61 (16.54)
Epoch 011/020 | Loss 0.5321 | Win/lose count 17.16/4.66 (12.5)
Epoch 012/020 | Loss 0.5437 | Win/lose count 8.33/1.84 (6.49)
Epoch 013/020 | Loss 0.5830 | Win/lose count 29.41/2.41 (27.0)
Epoch 014/020 | Loss 0.4526 | Win/lose count 16.19/2.68 (13.51)
Epoch 015/020 | Loss 0.7574 | Win/lose count 11.76

In [14]:
HTML(display_videos('cnn_train_explore19.mp4'))

In [15]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')

Win/lose count 12.74/1.74. Average score (11.0)
Win/lose count 6.37/1.88. Average score (7.745)
Win/lose count 14.23/1.72. Average score (9.33333333333)
Win/lose count 15.2/2.7. Average score (10.125)
Win/lose count 7.84/2.85. Average score (9.098)
Win/lose count 16.67/2.67. Average score (9.915)
Win/lose count 22.55/2.55. Average score (11.3557142857)
Win/lose count 5.88/1.89. Average score (10.435)
Win/lose count 11.28/2.78. Average score (10.22)
Win/lose count 19.11/3.59. Average score (10.75)
Win/lose count 19.13/1.61. Average score (11.3654545455)
Win/lose count 13.72/1.73. Average score (11.4175)
Win/lose count 15.68/1.68. Average score (11.6161538462)
Win/lose count 19.61/3.6. Average score (11.93)
Win/lose count 12.75/2.74. Average score (11.802)
Win/lose count 15.68/1.69. Average score (11.93875)
Win/lose count 3.92/1.93. Average score (11.3535294118)
Win/lose count 6.86/1.87. Average score (11.0)
Win/lose count 19.12/1.62. Average score (11.3421052632)
Win/lose count 7.36/2.8

In [18]:
HTML(display_videos('cnn_test_explore10.mp4'))

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***